In [1]:
!pip install BeautifulSoup4
import requests 
from bs4 import BeautifulSoup

     |████████████████████████████████| 97 kB 7.7 MB/s 
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
# XPATH_LIST_CONGRESSMEN = '//a[@class="conginfo"]/text()'
# XPATH_LIST_URLCONGINFO = '//a[@class="conginfo"]/@href'
# #in previous url
# XPATH_REPRESENT = '//p[@class="representa"]/span[@class="value"]/text()'
URLCONGRESSMEN = 'https://www.congreso.gob.pe/pleno/congresistas/'

In [3]:
#GET ALL NAMES AND URL'S
PLENO_PAGE = requests.get(URLCONGRESSMEN).text
PLENO_SOUP = BeautifulSoup(PLENO_PAGE, 'lxml')
LIST_PLENO = PLENO_SOUP.find_all(class_='conginfo')
LIST_CONGRESSMEN_NAMES = [i.text for i in LIST_PLENO]
LIST_URLCONGINFO = [i['href'] for i in LIST_PLENO]

In [ ]:
#GETTING REGIONS
LIST_REGIONES = []
LIST_VOTACION = []
for url in LIST_URLCONGINFO:
  CONG_PAGE = requests.get(URLCONGRESSMEN+url).text
  CONG_SOUP = BeautifulSoup(CONG_PAGE, 'lxml')
  LIST_REGIONES.append(CONG_SOUP.find('p', class_='representa').find(class_='value').text)
  LIST_VOTACION.append(CONG_SOUP.find('p', class_='votacion').find(class_='value').text)


In [ ]:
import pandas as pd
Congress = pd.DataFrame({"Congresista":LIST_CONGRESSMEN_NAMES, "Región":LIST_REGIONES, "Votación":LIST_VOTACION})
Congress['Comisiones'] = [[] for i in range(len(Congress['Congresista']))]

Primero se getea la pagina, después se convierte en sopa con BeautifulSoup (usando el atributo text y el atributo 'lxml') y después se usa el método find_all para buscar las cosas, después se usa el metodo get en cada elemento de la lista obtenida para obtener atributos, se puede usar el metodo get_text para obtener el texto

In [ ]:
urlcomisiones = 'https://www.congreso.gob.pe/CuadrodeComisiones/'
P_comisiones = requests.get(urlcomisiones)
P_comisiones.status_code


200

In [ ]:
soup_comisiones = BeautifulSoup(P_comisiones.text, 'lxml')
tags_comisiones = soup_comisiones.find_all('a', target='_blank')
comisiones = tags_comisiones[:24]

In [ ]:
links_comisiones = [i.get('href') for i in comisiones]
nombre_comisiones = [i.text for i in comisiones]


In [ ]:
n=0
COMISIONES_MIEMBROS_CARGOS = []
for URL_COMISION in links_comisiones:
  nonrawpage = requests.get(URL_COMISION+'/sobrelacomision/integrantes/')
  rawsoup = BeautifulSoup(nonrawpage.text, 'lxml')
  try:
    URL_COMISIONreal = rawsoup.find('iframe', attrs={'name':'ventana02'}).get('src')
  except:
    print("la interfaz no tiene los integrantes. Comision: "+nombre_comisiones[links_comisiones.index(URL_COMISION)])
  PaginaComision = requests.get(URL_COMISIONreal)
  SoupComision = BeautifulSoup(PaginaComision.text, 'lxml')
  Miembros = [i.get_text() for i in SoupComision.find_all('a', attrs={'class':'conginfo'})]
  Cargos = [ i.get_text() for i in SoupComision.find_all('span', attrs={'class':'partidolist'})][1::2]
  COMISIONES_MIEMBROS_CARGOS.append({Miembros[i]:[i[0] for i in Cargos][i] for i in range(len(Miembros))})
  print(COMISIONES_MIEMBROS_CARGOS[n])
  n+=1


https://www.congreso.gob.pe/?K=492&m23_idP=1094&TPL=if0
{'Olivos Martínez Vivian': 'P', 'Doroteo Carbajo Raúl Felipe': 'V', 'Díaz Monago Freddy Ronald': 'S', 'Castillo Rivas Eduardo Enrique': 'T', 'Chacón Trujillo Nilza Merly': 'T', 'Ciccia Vásquez Miguel Angel': 'T', 'Coayla Juárez Jorge Samuel': 'T', 'Córdova Lobatón María Jessica': 'T', 'García Correa Idelso Manuel': 'T', 'Gutiérrez Ticona Paul Silvio': 'T', 'Jeri Oré José Enrique': 'T', 'López Ureña Ilich Fredy': 'T', 'Palacios Huamán Margot': 'T', 'Taipe Coronado María Elizabeth': 'T', 'Ventura Angel Héctor José': 'T', 'Zea Choquechambi Oscar': 'T', 'Bustamante Donayre Ernesto': 'A', 'Cavero Alva Alejandro Enrique': 'A', 'Cordero Jon Tay Luis Gustavo': 'A', 'Flores Ancachi Jorge Luis': 'A', 'Guerra García Campos Hernando': 'A', 'Medina Minaya Esdras Ricardo': 'A', 'Montalvo Cubas Segundo Toribio': 'A', 'Morante Figari Jorge Alberto': 'A', 'Picón Quedo Luis Raúl': 'A', 'Revilla Villanueva César Manuel': 'A', 'Soto Palacios Wilson':

In [ ]:
for zip in COMISIONES_MIEMBROS_CARGOS:
  for (name, cargo) in zip.items():
        listindex=([nombre_comisiones[COMISIONES_MIEMBROS_CARGOS.index(zip)], cargo])
        Congress['Comisiones'][Congress.index[Congress['Congresista']==name].tolist()[0]].append(listindex)


In [ ]:
#remember that response is the variable that contains URLCONGRESSMEN
soup_partido = BeautifulSoup(response.text, 'lxml')
partidos = [i.get_text() for i in soup_partido.find_all('span', attrs={'class':'partidolist'})]
Congress['Bancada'] = partidos
Congress.to_csv('Congress.csv', index=False)

In [ ]:
Congress['Votación'].hist()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=496b7f65-72e8-4f99-acc9-201d5fae973f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>